## Prostate MRI Dataset

* Cross-validation by patient
* Load feature map with ADC, CDI, Zones


In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
from PIL import Image
import numpy as np

import scipy.io
import os
import matplotlib.pyplot as plt
import pickle

%run -i LoadZoneData

In [2]:
# Load the data
save_filename = 'raw_data'

dir_path = os.path.join('.','data', 'data','data')
#data = import_data(dir_path)
#save_obj(data, save_filename)

# Drop these patients (incomplete data)

#drop_patients = ['P00000015', 'P00000249', 'P00000429']
#drop_ids = []
#for i,e in reversed(list(enumerate(data))):
#    if data[i]['id'] in drop_patients:
#        print(i)
#        del data[i]

# Resave the data object without those patients        
#save_obj(data, save_filename)
data = load_obj(save_filename)

In [43]:
# define the dictionary of models our script can use, where the key
# to the dictionary is the name of the model (supplied via command
# line argument) and the value is the model itself
models = {
	"knn": KNeighborsClassifier(n_neighbors=1),
	"naive_bayes": GaussianNB(),
	"logit": LogisticRegression(solver="lbfgs"),
	"svm": SVC(kernel="linear"),
	"decision_tree": DecisionTreeClassifier(),
	"random_forest": RandomForestClassifier(n_estimators=100),
	"mlp": MLPClassifier()
}

In [5]:
fold_dict = {}
with open('5folds.txt', 'r') as f:
    for line in f:
        pid, fold_no = line.split()
        fold_dict[str(pid)] = int(fold_no)

## Data Preprocessing and Feature Extraction

In [36]:
examples = {}
labels = []

num_features = 2
#patient = data[0]
for modality in ['ADC']: # TODO: add T2-weighted images (if we get the labels)
    
    if modality == 'ADC':
        label_map = 'maxGleason_map'
        zone_map = 'zone_map'
        
    examples[modality] = []
    
    for _ in range(5):
        examples[modality].append([])
        labels.append([])

    for i in range(5): 
        for _ in range(num_features+2): # num features (1 mod, 1 zones)
            examples[modality][i].append([])
        
    for patient in data:

        pid = patient['id']
        if pid in ['P00000015', 'P00000249', 'P00000429']: # remove bad data
            continue

        fold_id = fold_dict[pid] - 1

        patient_examples = []
        patient_labels = []

        if patient[zone_map].shape[-1] != patient[modality].shape[-1]: # check if segmentation map has same num slices as mri
            continue

        for slice_index in range(patient[modality].shape[-1]):
            
                # Check if the slice contains the prostate
            if np.max(patient['mask'][:,:,slice_index]) > 0:
                z_map = patient[zone_map][:,:,slice_index] > 0
                trimmed_zone_map,indexes = trim_zeros_2D(z_map)
                mod_slice = (patient[modality][:,:,slice_index] * z_map)[indexes[0]:indexes[1]+1,indexes[2]:indexes[3]+1]

                # Perform slice-level feature extraction
                features = feat_extraction_1d(mod_slice)
                features.append(np.array(mod_slice))
                
                
                
                for zone_index in range(10):
                    zone_number = zone_index + 1
                    if zone_number in patient[zone_map][:,:,slice_index]: # check zone map to see if the slice contains the zone
                        binary_mask = (patient[zone_map][:,:,slice_index] * z_map)[indexes[0]:indexes[1]+1,indexes[2]:indexes[3]+1] == zone_number  # create a binary mask

                        # Apply masks to slices (mod and features) 
                        feat_zone = features * binary_mask # apply the mask to the slice
                        trimmed_mod_zone, idx = trim_zeros_2D(mod_slice)
                        for f in range(len(feat_zone)):
                            trimmed_f_zone = feat_zone[f][idx[0]:idx[1]+1,idx[2]:idx[3]+1]# trim the slice to the dimensions of the zone num
                            #patient_examples[f].extend(trimmed_f_zone)
                            for item in trimmed_f_zone:
                                examples[modality][fold_id][f].extend(np.array(item))
                            label = [(1 if patient[label_map][slice_index][zone_index] >0 else 0)]*len(trimmed_f_zone[0])*len(trimmed_mod_zone)
                        
                        examples[modality][fold_id][len(feat_zone)].extend([zone_number] * len(trimmed_mod_zone) * len(trimmed_mod_zone[0])) # Add zone as a feature
                        
                        patient_labels.extend(label)
                        
                        #patient_labels.append(1 if patient[label_map][slice_index][zone_index] >0 else 0)
                        
        #examples[modality][fold_id].extend(patient_examples)
        labels[fold_id].extend(patient_labels)
        
    

In [37]:
print(len(examples['ADC'][1]))
print(len(labels[1]))

4
1781655


## Script to run the model, training and prediction

In [38]:
def predict_and_report(model):
    modality = 'ADC'
    aucs = []
    rus = RandomUnderSampler(random_state=0)

    for i in range(5): # CV Loop
        x_train = []
        y_train = []

        x_test = []
        y_test = labels[i]
        
        for _ in range(len(examples[modality][i])):
            x_train.append([])
            x_test.append([])
        
        # Training on these
        for j in range(5):
            if i != j:
                for f in range(len(examples[modality][j])):
                    x_train[f].extend(examples[modality][j][f])
                y_train.extend(labels[j])
            else:
                for f in range(len(examples[modality][i])):
                    x_test[f].extend(examples[modality][i][f])
        
        x_train = np.array(x_train).T
        x_test = np.array(x_test).T
        
        # TODO: feature extraction / features selection / classification here
        
        # Undersampling, balancing
        x_rus, y_rus = rus.fit_sample(x_train, y_train)
        
        # Standardize
        print('Standardizing...')
        sc = StandardScaler().fit(x_rus)

        stand_X = sc.transform(x_rus)
        stand_X_test = sc.transform(x_test)
        
        # Train Model
        print('Training on model... ' + model + ' - Run ' + str(i+1))
        
        x_df = pd.DataFrame(stand_X)
        
        mod = models[model]
        mod.fit(x_df, y_rus)
                
        print('Evaluating on model... ' + model + ' - Run ' + str(i+1))
                
        y_pred = mod.predict(stand_X_test)
                
        auc = roc_auc_score(y_test, y_pred)
        aucs.append(auc)
        print('AUC: ' + auc)
        print(confusion_matrix(y_test, y_pred))
        print(classification_report(y_test,y_pred))
        # auc = auc_calculation(y_pred, y_test)

    mean_auc = sum(aucs) / float(len(aucs))
    print(mean_auc)

In [46]:
predict_and_report('decision_tree')

Standardizing...
Training on model... decision_tree - Run 1
Evaluating on model... decision_tree - Run 1
0.7277232542721817
[[1114739  810097]
 [   2133   15112]]
             precision    recall  f1-score   support

          0       1.00      0.58      0.73   1924836
          1       0.02      0.88      0.04     17245

avg / total       0.99      0.58      0.73   1942081

Standardizing...
Training on model... decision_tree - Run 2
Evaluating on model... decision_tree - Run 2
0.718652516919297
[[1037638  728718]
 [   2297   13002]]
             precision    recall  f1-score   support

          0       1.00      0.59      0.74   1766356
          1       0.02      0.85      0.03     15299

avg / total       0.99      0.59      0.73   1781655

Standardizing...
Training on model... decision_tree - Run 3
Evaluating on model... decision_tree - Run 3
0.6552531449746817
[[922504 623339]
 [  4437  11063]]
             precision    recall  f1-score   support

          0       1.00      0.60